# LLM Fine-Tuning using LoRA

This notebook provides an example on how to use LoRA to improve a LLM performance on a specific task, such as sentiment analysis.!pip install torch
!pip install datasets transformers sentencepiece
!pip install tqdm
!pip install sacrebleu

In [ ]:
!pip install torch
!pip install datasets transformers evaluate sentencepiece
!pip install tqdm
!pip install peft

# 0. Libraries, constants and support functions 

In [1]:
import torch
import evaluate
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType

2024-10-08 20:57:16.406417: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# 1. Obtain performance from pre-trained model

In [2]:
# Load imbd dataset
imdb_dataset = load_dataset('imdb')

# Split between train/test
imdb_dataset_train = imdb_dataset['train']
imdb_dataset_test = imdb_dataset['test']

In [3]:
# Load tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

# Tokenize dataset
tokenized_imdb_train = imdb_dataset_train.map(preprocess_function, batched=True, remove_columns=['text'])
tokenized_imdb_test = imdb_dataset_test.map(preprocess_function, batched=True, remove_columns=['text'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [5]:
# Convert to torch dataset format
tokenized_imdb_train.set_format('torch')
tokenized_imdb_test.set_format('torch')

In [6]:
# Define performance function
accuracy_metric = evaluate.load('accuracy')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    # Obtain output and labels
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) # Apply argmax
    # Apply metrics and add to dictionary
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)['accuracy']
    precision = precision_metric.compute(predictions=predictions, references=labels, average='binary')['precision']
    recall = recall_metric.compute(predictions=predictions, references=labels, average='binary')['recall']
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='binary')['f1']
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}


In [7]:
# Define training parameters
training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=64,
)

# Define trainer with parameters
pre_trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_imdb_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [8]:
print("Evaluating the pre-trained model before fine-tuning...")
pre_eval_results = pre_trainer.evaluate()
print(pre_eval_results)

Evaluating the pre-trained model before fine-tuning...


/opt/conda/envs/python311/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.6998547911643982, 'eval_accuracy': 0.5062, 'eval_precision': 0.5032701801763787, 'eval_recall': 0.95416, 'eval_f1': 0.6589684797922595, 'eval_runtime': 180.4259, 'eval_samples_per_second': 138.561, 'eval_steps_per_second': 1.086}


This accuracy is pretty bad, to be honest...

Let's see if we can fix this by fine-tuning the model

# 2. Fine-tune the model with LoRA

In [10]:
# Define LoRA strategy
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

# Attach LoRA to model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700


In [13]:
# Create trainer for Fine-Tuning
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb_train,
    eval_dataset=tokenized_imdb_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

/opt/conda/envs/python311/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
print("Fine-tuning the model...")
trainer.train()

Fine-tuning the model...


/opt/conda/envs/python311/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


/opt/conda/envs/python311/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [ ]:
print("Evaluating the fine-tuned model...")
eval_results = trainer.evaluate()
print(eval_results)